In [ ]:
import asmsa
import mdtraj as md
import nglview as nv
import numpy as np
from tensorflow.keras.saving import load_model
import torch
import gromacs as gmx
from gromacs import formats as gmf
import matplotlib.pyplot as plt

In [ ]:
exec(open('inputs.py').read())

In [ ]:
test_tr = md.load('test.xtc',top=conf)
train_tr = md.load('train.xtc',top=conf)

In [ ]:
test_g = np.moveaxis(test_tr.xyz,0,-1)
train_g = np.moveaxis(train_tr.xyz,0,-1)

In [ ]:
sparse_dists = asmsa.NBDistancesSparse(train_g.shape[0], density=2)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

In [ ]:
with open('dih4.ndx','w') as a:
    a.write('[ dihedrals ]\n')
    for d in mol.dihed4+1:
        a.write('  '.join(map(str,d)))
        a.write('\n')

In [ ]:
gmx.gangle(f='test.xtc',n='dih4.ndx',g1='dihedral',group1='dihedrals',oall='dih4.xvg')

In [ ]:
dihs = gmf.XVG('dih4.xvg').array[1:] * np.pi / 180.

In [ ]:
dihs.shape

In [ ]:
ic = mol.intcoord(test_g)
ic.shape

In [ ]:
off = mol.angles.shape[0]+mol.bonds.shape[0]
ndih4 = mol.dihed4.shape[0]

In [ ]:
ic[off]

In [ ]:
a = 27
plt.plot(np.sin(dihs[a])[::100],label='gmx sin')
plt.plot(ic[a+off][::100],label='ic sin')

plt.legend()
plt.show()

In [ ]:
plt.plot(np.cos(dihs[a])[::100],label='gmx cos')
plt.plot(ic[a+off+ndih4][::100],label='ic cos')
plt.legend()
plt.show()

In [ ]:
dih_ic = np.arctan2(ic[off:off+ndih4],ic[off+ndih4:off+2*ndih4])

In [ ]:
diff = dihs - dih_ic
diff[diff >= 2*np.pi] -= 2*np.pi
diff[diff <= -2*np.pi] += 2*np.pi

In [ ]:
np.max(np.abs(diff))

In [ ]:
with open('angle.ndx','w') as a:
    a.write('[ angles ]\n')
    for d in mol.angles + 1:
        a.write('  '.join(map(str,d)))
        a.write('\n')

In [ ]:
gmx.gangle(f='test.xtc',n='angle.ndx',g1='angle',group1='angles',oall='angles.xvg')

In [ ]:
angles = gmf.XVG('angles.xvg').array[1:] * np.pi / 180.

In [ ]:
aoff = mol.bonds.shape[0]

In [ ]:
a = 28
plt.plot(np.cos(angles[a])[::100],label='gmx cos')
plt.plot(ic[a+aoff][::100],label='ic cos')

plt.legend()
plt.show()

In [ ]:
np.unique(mol.angles,axis=0).shape, mol.angles.shape

In [ ]:
np.unique(mol.dihed4,axis=0).shape, mol.dihed4.shape

In [ ]:
np.unique(mol.dihed9,axis=0).shape, mol.dihed9.shape

In [ ]:
np.unique(mol.dihed9,axis=0)